Case Study #2<br>

There is 1 dataset(csv) with 3 years worth of customer orders. There are 4 columns in the csv dataset: index, CUSTOMER_EMAIL(unique identifier as hash), Net_Revenue, and Year.
For each year we need the following information:
- Total revenue for the current year
- New Customer Revenue e.g. new customers not present in previous year only
- Existing Customer Growth. To calculate this, use the Revenue of existing customers for current year –(minus) Revenue of existing customers from the previous year
- Revenue lost from attrition
- Existing Customer Revenue Current Year
- Existing Customer Revenue Prior Year
- Total Customers Current Year
- Total Customers Previous Year
- New Customers
- Lost Customers
<br>
Additionally, generate a few unique plots highlighting some information from the dataset. Are there any interesting observations?


In [1]:
import pandas as p
import scipy.stats
import numpy as n
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
import time
from itertools import permutations
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline


In [2]:
data = p.read_csv("casestudy.csv")

In [3]:
data.head()

,Unnamed: 0,customer_email,net_revenue,year
0,0,nhknapwsbx@gmail.com,249.92,2015
1,1,joiuzbvcpn@gmail.com,87.61,2015
2,2,ukkjctepxt@gmail.com,168.38,2015
3,3,gykatilzrt@gmail.com,62.40,2015
4,4,mmsgsrtxah@gmail.com,43.08,2015


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 685927 entries, 0 to 685926
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      685927 non-null  int64  
 1   customer_email  685927 non-null  object 
 2   net_revenue     685927 non-null  float64
 3   year            685927 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 20.9+ MB


In [68]:
data.shape

(685927, 6)

In [5]:
data['customer_email'].value_counts()

wtbxliuywf@gmail.com     3
leafpkocos@gmail.com     3
yutbpprfrj@gmail.com     3
pwllhvbpzv@gmail.com     3
ofysynyagn@gmail.com     3
                        ..
 jzkxvcumhr@gmail.com    1
gknvzmmwxi@gmail.com     1
qjxzeiurgm@gmail.com     1
 srhiqoyxrk@gmail.com    1
 aregboumbw@gmail.com    1
Name: customer_email, Length: 604618, dtype: int64

In [6]:
data['customer_email'] = data['customer_email'].str.strip()

In [7]:
data['net_revenue'].value_counts()

219.89    49
166.72    49
51.33     48
236.04    48
198.61    48
          ..
62.88     11
180.32    11
126.29    11
132.67    10
186.22    10
Name: net_revenue, Length: 24901, dtype: int64

In [8]:
data['year'].value_counts()

2017    249987
2015    231294
2016    204646
Name: year, dtype: int64

## Total revenue for current year (Assuming 2017 as current year)

In [59]:
data.groupby('year').net_revenue.sum().iloc[-1]

31417495.03

## New Customer Revenue e.g. new customers not present in previous year only

In [10]:
cust_year_revenue = data.groupby(['customer_email','year']).net_revenue.sum()
cust_year_revenue = cust_year_revenue.reset_index().sort_values(by = ['customer_email','year'])
df_16 = cust_year_revenue[cust_year_revenue['year']==2016]

In [11]:
df_17 = cust_year_revenue[cust_year_revenue['year']==2017]

In [12]:
df_16.merge(df_17, on = 'customer_email',how = 'right')

,customer_email,year_x,net_revenue_x,year_y,net_revenue_y
0,aaaaaphwwy@gmail.com,NaN,NaN,2017,241.10
1,aaafxtkgxo@gmail.com,2016.0,127.62,2017,242.28
2,aaaiekepot@gmail.com,NaN,NaN,2017,43.68
3,aaaiexlzvq@gmail.com,NaN,NaN,2017,2.58
4,aaakcnoovj@gmail.com,NaN,NaN,2017,72.22
...,...,...,...,...,...
249982,zzzmkaruhf@gmail.com,NaN,NaN,2017,202.77
249983,zzznqseiwv@gmail.com,NaN,NaN,2017,162.24
249984,zzzoxtrfic@gmail.com,NaN,NaN,2017,15.74
249985,zzztmtdlbv@gmail.com,NaN,NaN,2017,81.30


In [13]:
cust_year_revenue['cust_revenue_prev_yr'] = cust_year_revenue.groupby(['customer_email']).net_revenue.shift()

In [14]:
cust_year_revenue['cust_revenue_prev_2_yr'] = cust_year_revenue.groupby(['customer_email']).net_revenue.shift(2)

In [15]:
cust_year_revenue.to_csv("cust_year_revenue.csv")

In [16]:
cust_year_revenue[cust_year_revenue['customer_email']=='aaafxtkgxo@gmail.com']

,customer_email,year,net_revenue,cust_revenue_prev_yr,cust_revenue_prev_2_yr
4,aaafxtkgxo@gmail.com,2016,127.62,NaN,NaN
5,aaafxtkgxo@gmail.com,2017,242.28,127.62,NaN


In [17]:
cust_year_revenue[cust_year_revenue['customer_email']=='zzztmtdlbv@gmail.com']

,customer_email,year,net_revenue,cust_revenue_prev_yr,cust_revenue_prev_2_yr
685920,zzztmtdlbv@gmail.com,2017,81.3,NaN,NaN


In [18]:
cust_year_revenue['new_customer_yearwise'] = n.where(~p.isnull(cust_year_revenue['net_revenue'])\
                                                      & p.isnull(cust_year_revenue['cust_revenue_prev_yr'])\
                                                     & p.isnull(cust_year_revenue['cust_revenue_prev_2_yr']), 'New','Old')

In [19]:
new_old_revenue = cust_year_revenue[cust_year_revenue['year']!=2015].groupby(['year','new_customer_yearwise']).net_revenue.sum().unstack()
new_old_revenue

new_customer_yearwise,New,Old
year,,
2016,17206366.90,8524576.69
2017,21769213.01,9648282.02


## Existing Customer Growth. To calculate this, use the Revenue of existing customers for current year –(minus) Revenue of existing customers from the previous year

In [20]:
new_old_revenue.loc['Growth',:] = new_old_revenue.loc[2017,:] - new_old_revenue.loc[2016,:] 
new_old_revenue

new_customer_yearwise,New,Old
year,,
2016,17206366.90,8524576.69
2017,21769213.01,9648282.02
Growth,4562846.11,1123705.33


In [21]:
new_old_revenue.loc['Growth','Old']

1123705.33

## Revenue lost from attrition

In [22]:
# p.crosstab(data['year'],data['customer_email'])
customers_churned_15_17 = p.pivot_table(data = cust_year_revenue ,columns = 'year',index = 'customer_email',aggfunc = n.sum ,values = 'net_revenue')
customers_churned_15_17

year,2015,2016,2017
customer_email,,,
aaaaaphwwy@gmail.com,NaN,NaN,241.10
aaabbviidb@gmail.com,133.07,NaN,NaN
aaaewsybit@gmail.com,184.55,NaN,NaN
aaafiismkd@gmail.com,113.28,NaN,NaN
aaafxtkgxo@gmail.com,NaN,127.62,242.28
...,...,...,...
zzzuzmfsii@gmail.com,NaN,230.79,NaN
zzzwmlwwmn@gmail.com,NaN,98.01,NaN
zzzxxclrua@gmail.com,57.24,NaN,NaN


In [23]:
customers_churned_15_17[~p.isnull(customers_churned_15_17[2015]) \
                        & p.isnull(customers_churned_15_17[2016]) & \
                        p.isnull(customers_churned_15_17[2017])]

year,2015,2016,2017
customer_email,,,
aaabbviidb@gmail.com,133.07,NaN,NaN
aaaewsybit@gmail.com,184.55,NaN,NaN
aaafiismkd@gmail.com,113.28,NaN,NaN
aaagldjgdt@gmail.com,44.72,NaN,NaN
aaaicvtnzx@gmail.com,49.20,NaN,NaN
...,...,...,...
zzzklahhms@gmail.com,134.25,NaN,NaN
zzzrjsowtl@gmail.com,95.80,NaN,NaN
zzztwcvgvp@gmail.com,129.75,NaN,NaN


## Revenue lost from attrition

In [24]:
#revenue lost due to customers not purchasing back in 2016 and 2017. 
#Assuming they would buy have same amount of purchase in both years

customers_churned_15_17[~p.isnull(customers_churned_15_17[2015]) &\
                        p.isnull(customers_churned_15_17[2016]) & \
                        p.isnull(customers_churned_15_17[2017])][2015].sum()*2

41102432.3

In [25]:
customers_churned_15_17[~p.isnull(customers_churned_15_17[2015]) & \
                        ~p.isnull(customers_churned_15_17[2016]) & \
                        p.isnull(customers_churned_15_17[2017])][2016].sum()

5350184.100000001

In [26]:
#attrition is when you joined in some year and after that never ordered again in next year
#when the customer has not interacted with or purchased from the company.

##Ways to do??
#2015 mein purchase kiya but 2016 and 2017 mein nahi kiya
#2016 mein purchase kiya but 2017 mein nahi kiya

#purchase per year for customer
#churn year


## Existing Customer Revenue Current Year

In [27]:
new_old_revenue.loc[2017,'Old']

9648282.02

## Existing Customer Revenue Prior Year

In [28]:
new_old_revenue.loc[2016,'Old']

8524576.69

## Total Customers Current Year

In [29]:
cust_year_revenue[(cust_year_revenue['year'] == 2017)&\
                  (~p.isnull(cust_year_revenue['net_revenue']))]['customer_email'].count()

249987

## Total Customers Previous Year

In [30]:
# I assumed that previous customers mean only those customers who did some purchasing in 2016
cust_year_revenue[(cust_year_revenue['year'] == 2016)&\
                  (~p.isnull(cust_year_revenue['net_revenue']))]['customer_email'].count()

204646

## New Customers

In [62]:
new_old_count = cust_year_revenue[cust_year_revenue['year']!=2015]\
.groupby(['year','new_customer_yearwise']).customer_email.count().unstack()
new_old_count
new_old_count['percent'] = new_old_count['New']/(new_old_count['New']+new_old_count['Old'])*100
new_old_count

new_customer_yearwise,New,Old,percent
year,,,
2016,136891,67755,66.891608
2017,173449,76538,69.383208


In [32]:
new_old_count['New']

year
2016    136891
2017    173449
Name: New, dtype: int64

## Old Customers

In [33]:
new_old_count['Old']

year
2016    67755
2017    76538
Name: Old, dtype: int64

## Lost Customers

In [64]:
customers_churned_15_17

year,2015,2016,2017,%change,revenue_cat
customer_email,,,,,
aaaaaphwwy@gmail.com,NaN,NaN,241.10,NaN,<=125
aaabbviidb@gmail.com,133.07,NaN,NaN,NaN,>125
aaaewsybit@gmail.com,184.55,NaN,NaN,NaN,>125
aaafiismkd@gmail.com,113.28,NaN,NaN,NaN,<=125
aaafxtkgxo@gmail.com,NaN,127.62,242.28,NaN,<=125
...,...,...,...,...,...
zzzuzmfsii@gmail.com,NaN,230.79,NaN,NaN,<=125
zzzwmlwwmn@gmail.com,NaN,98.01,NaN,NaN,<=125
zzzxxclrua@gmail.com,57.24,NaN,NaN,NaN,<=125


## Lost Customers

In [67]:
#Assuming customers which were present in 2015, 2016 but were lost in 2017
customers_churned_15_17[~p.isnull(customers_churned_15_17[2015]) & \
                        ~p.isnull(customers_churned_15_17[2016]) & \
                        p.isnull(customers_churned_15_17[2017])].reset_index()['customer_email'].count()

42469